<a href="https://colab.research.google.com/github/mhtsos99/BlackJack/blob/main/RBC%20criterion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ============================================================
# RBC TOOLBOX (Beginner-friendly version with many comments)
# ============================================================
# What this script does:
# - It helps a developer check if they are ready for the RBC (Responsible Business Conduct) criterion
# - It follows the logic of Article 4 from:
#   Commission Implementing Regulation (EU) 2025/1176 (23 May 2025)
#
# How the script works (high level):
# 1) Ask if RBC is included in the auction + if the auction follows Regulation 2025/1176
#    - If not, stop early (no errors / no traceback)
# 2) Ask project capacity (MW)
#    - If <10 MW, the toolbox stops (you can skip RBC due to size rule)
# 3) If >=10 MW, ask if you are Option A or Option B
# 4) Depending on Option A/B, ask questions and show what is missing
# 5) Print final results as a 2-column table
# ============================================================


# --------------------------
# IMPORTS (libraries)
# --------------------------

# We import pandas because it helps us make a nice table at the end.
# pandas = a library for working with tables/data (DataFrames).
import pandas as pd

# "display" is a nice function inside Jupyter/Colab notebooks that shows tables nicely.
# BUT: if you run this in a normal Python terminal, display() may not exist.
# So we try to import it, and if it fails, we create our own display() that just prints.
try:
    from IPython.display import display  # type: ignore
except Exception:
    # If IPython isn't available, we define a "display" function ourselves
    # so our code still works everywhere.
    def display(x):
        print(x)


# --------------------------
# CONSTANTS (fixed info)
# --------------------------

# These are fixed strings (constants). We keep them at the top so we can change them easily later.
REGULATION_NAME_SHORT = "Commission Implementing Regulation (EU) 2025/1176"
REGULATION_DATE = "(23 May 2025)"

# Links where the user can check the original legal texts:
LINK_RBC_ARTICLE4 = "https://eur-lex.europa.eu/eli/reg_impl/2025/1176/oj/eng"
LINK_CSDDD = "https://eur-lex.europa.eu/eli/dir/2024/1760/oj/eng"
LINK_DELEGATED_2772 = "https://eur-lex.europa.eu/eli/reg_del/2023/2772/oj/eng"


# ============================================================
# HELPER FUNCTIONS
# ============================================================
# Why we use helper functions:
# - To avoid repeating code many times
# - To make input handling consistent and safer
# - To keep the main logic readable
# ============================================================

def _ask_yes_no(prompt: str) -> bool:
    """
    Ask a yes/no question and return True for yes, False for no.

    We use a while loop so the user MUST answer correctly.
    This prevents crashes or bad inputs.
    """
    while True:
        ans = input(prompt).strip().lower()  # .strip() removes spaces, .lower() makes it lowercase
        if ans in {"yes", "y"}:
            return True
        if ans in {"no", "n"}:
            return False
        print("Please answer with 'yes' or 'no'.")


def _ask_float(prompt: str, min_val=None) -> float:
    """
    Ask for a number (float).
    We use try/except because if the user types text, float(...) would crash.
    """
    while True:
        try:
            val = float(input(prompt).strip())
            if min_val is not None and val < min_val:
                print(f"Value must be >= {min_val}.")
                continue
            return val
        except ValueError:
            print("Please enter a valid number.")


def _ask_option_a_b(prompt: str) -> str:
    """
    Ask for Option A or Option B.
    Returns "a" or "b" (always lowercase).
    """
    while True:
        ans = input(prompt).strip().lower()
        if ans in {"a", "b"}:
            return ans
        print("You wrote something else. Please type only 'A' or 'B' (or 'a'/'b').")


def _parse_multi_letters(raw: str, allowed: set[str]) -> list[str]:
    """
    Convert input like:
      "a,b,c" OR "a b c" OR "A, C"
    into a list like ["a", "c"] (only valid letters, unique, sorted).

    Steps:
    - Replace spaces with commas
    - Split by commas
    - Keep only tokens that are inside allowed set
    - Remove duplicates using set()
    """
    tokens = raw.replace(" ", ",").split(",")
    cleaned = []

    for t in tokens:
        t = t.strip().lower()
        if t in allowed:
            cleaned.append(t)

    return sorted(set(cleaned))


def _ask_missing_letters(prompt: str, allowed_letters: list[str], allow_empty: bool = True) -> list[str]:
    """
    Ask which letters are missing (multiple answers allowed).
    Example: a,b,c

    If user types invalid input, we ask again.

    Returns list like ["a", "c"].
    """
    allowed_set = set([x.lower() for x in allowed_letters])

    while True:
        raw = input(prompt).strip()

        # If empty input is allowed and they typed nothing, return empty list.
        if raw == "" and allow_empty:
            return []

        selected = _parse_multi_letters(raw, allowed_set)

        # If they typed something but nothing valid, ask again.
        if raw != "" and len(selected) == 0:
            print(f"Please type letters from: {', '.join(allowed_letters)} (example: a,b,c).")
            continue

        return selected


def _format_letter_list(letters: list[str]) -> str:
    """
    Format list for nice English printing.
    Example:
      ["a"] -> "a"
      ["a","b"] -> "a and b"
      ["a","b","c"] -> "a, b and c"
    """
    if not letters:
        return ""
    if len(letters) == 1:
        return letters[0]
    if len(letters) == 2:
        return f"{letters[0]} and {letters[1]}"
    return ", ".join(letters[:-1]) + f" and {letters[-1]}"


# ============================================================
# RBC TOOLBOX — Step 1 — Inputs (User interaction)
# ============================================================

print("\n============================================================")
print("RBC toolbox (focus: only the RBC criterion)")
print(f"Runs according to: {REGULATION_NAME_SHORT} {REGULATION_DATE}")
print("RBC is mandatory in this EU law with specific requirements. This toolbox follows exactly that.")
print("============================================================\n")

# 1) Ask if RBC is included in the auction
rbc_in_auction = _ask_yes_no("Is the RBC criterion included in your auction? (yes/no): ")

# If RBC is NOT included => stop early (we do NOT use sys.exit to avoid notebook traceback)
if not rbc_in_auction:
    print("\nRBC is not included in your auction. You can skip this criterion.")
else:
    # 2) Ask if auction follows Regulation 2025/1176 for RBC
    auction_follows_1176 = _ask_yes_no("Does your auction follow Regulation (EU) 2025/1176 for RBC? (yes/no): ")

    # If auction does not follow the regulation => stop early
    if not auction_follows_1176:
        print("\nThis toolbox currently cannot help you with this criterion if your auction does not follow Regulation (EU) 2025/1176.")
    else:
        # Continue only if both answers are YES

        # Show where user can read original Article 4
        print("\n--- RBC criterion original text ---")
        print("You can read the original text for the RBC criterion (Article 4) here:")
        print(LINK_RBC_ARTICLE4)
        print("Tip: on the HTML page, use Ctrl+F and search: 'Article 4'.\n")

        # Ask project capacity
        project_capacity_mw = _ask_float("Project capacity (MW): ", min_val=0.0)

        # If capacity < 10 MW => toolbox stops (as you requested)
        if project_capacity_mw < 10:
            print("\nAccording to Commission Implementing Regulation (EU) 2025/1176,")
            print("projects below 10 MW are fully exempt from this RBC criterion due to their size.")
            print("You can skip this criterion.")
        else:
            # If capacity >=10 MW => ask for Option A or B
            print("\n--- Choose bidder category (According to Article 4) ---")
            print("Option A: Natural persons OR companies outside Directive 2013/34/EU (Arts 19a & 29a) OR renewable energy communities")
            print("Option B: Standard company")
            option = _ask_option_a_b("Type A or B: ")

            # We create these variables to store results and missing items
            missing_items = []
            note = ""

            # This is only for the final results (text label)
            bidder_type_label = "Option A = small actor" if option == "a" else "Option B = standard/big actor"

            # ============================================================
            # RBC TOOLBOX — Step 2 — Calculations / checks
            # ============================================================

            def _csddd_ag_check() -> list[str]:
                """
                Check Article 5(1)(a)-(g) documentation.
                - If user says YES => return []
                - If NO => ask which letters are missing (a-g) and return them.
                """
                has_ag = _ask_yes_no("Do you have documentation covering ALL items (a)-(g)? (yes/no): ")
                if has_ag:
                    return []
                miss = _ask_missing_letters(
                    "Which items are missing? Type letters separated by commas (a,b,c,d,e,f,g): ",
                    allowed_letters=["a", "b", "c", "d", "e", "f", "g"],
                    allow_empty=False
                )
                return miss

            def _annex_61ae_check() -> list[str]:
                """
                Check public statement coverage for Annex I point 61(a)-(e).
                - If YES => []
                - If NO => ask which letters are missing (a-e)
                """
                has_61ae = _ask_yes_no("Do you have that public RBC statement covering 61(a)-(e)? (yes/no): ")
                if has_61ae:
                    return []
                miss = _ask_missing_letters(
                    "Which elements are missing? Type letters separated by commas (a,b,c,d,e): ",
                    allowed_letters=["a", "b", "c", "d", "e"],
                    allow_empty=False
                )
                return miss

            def _light_61cd_check() -> list[str]:
                """
                Light route check (Option A default, Article 4(3)):
                - You need 61(c)+61(d) OR a voluntary standard.
                - If YES => []
                - If NO => ask what is missing among c, d, v.
                """
                has_light_pack = _ask_yes_no(
                    "Do you have the required documents for (61c + 61d) OR an EU voluntary standard (if available)? (yes/no): "
                )
                if has_light_pack:
                    return []

                print("Please specify what is missing (you can give multiple):")
                print("  - c  = 61(c) due-diligence process used")
                print("  - d  = 61(d) impacts identified + actions taken")
                print("  - v  = EU voluntary sustainability reporting standard (if available)")

                miss_letters = _ask_missing_letters(
                    "Type letters separated by commas (c,d,v): ",
                    allowed_letters=["c", "d", "v"],
                    allow_empty=False
                )

                # We map c/d/v to the full text we want to show in results.
                mapping = {
                    "c": "Annex I 61(c): due-diligence process used",
                    "d": "Annex I 61(d): impacts identified + actions taken",
                    "v": "EU voluntary sustainability reporting standard (where available)",
                }
                return [mapping[x] for x in miss_letters]

            def evaluate_rbc_readiness_for_option_a():
                """
                This function handles Option A logic:
                - Article 4(3) = light reporting by default
                - Article 4(4) = Member State may apply FULL obligations instead
                """
                local_missing = []

                print("\n--- Option A requirements ---")
                print("According to Article 4(3): you should report only on Annex I 61(c) and 61(d),")
                print("OR use EU voluntary sustainability reporting standards (if available).")
                print("However, Member States may instead apply FULL obligations (Article 4(4)).\n")

                # Ask if Member State applies FULL obligations
                ms_full = _ask_yes_no("Has the Member State applied FULL obligations to you (instead of light reporting)? (yes/no): ")

                # If MS does NOT apply full obligations => Light route
                if not ms_full:
                    print("\nLight reporting (Article 4(3)) requires:")
                    print("- 61(c): due-diligence process used")
                    print("- 61(d): impacts identified + actions taken")
                    print("OR EU voluntary sustainability reporting standards (if available).")
                    print("Link (to check yourself):", LINK_DELEGATED_2772, "\n")

                    light_missing = _light_61cd_check()
                    if light_missing:
                        local_missing.extend(light_missing)

                    status = "Completed" if len(local_missing) == 0 else "Incomplete"
                    return status, "Option A: light reporting route.", local_missing

                # If MS DOES apply full obligations => same as Option B checks
                print("\nAccording to Article 4(1), you must cover ALL due-diligence elements in CSDDD Article 5(1)(a)-(g):")
                print(
                    """Article 5(1) actions:
(a) integrate due diligence into policies and risk management systems
(b) identify and assess actual/potential adverse impacts (and prioritise where needed)
(c) prevent/mitigate potential impacts; bring actual impacts to an end and minimise extent
(d) provide remediation for actual adverse impacts
(e) meaningful engagement with stakeholders
(f) notification mechanism + complaints procedure
(g) monitor effectiveness of due diligence policy and measures
"""
                )
                print("Link:", LINK_CSDDD)

                miss_ag = _csddd_ag_check()
                if miss_ag:
                    local_missing.append(f"CSDDD Article 5(1) missing: ({_format_letter_list(miss_ag)})")

                print("\nAlso required (according to Article 4(2)): publish a public statement covering Annex I point 61(a)-(e).")
                print(
                    """Core elements (Annex I 61(a)-(e)):
(a) embedding due diligence in governance, strategy and business model
(b) engaging with affected stakeholders
(c) identifying and assessing negative impacts on people and the environment
(d) taking action to address negative impacts on people and the environment
(e) tracking the effectiveness of these efforts
"""
                )
                print("Link:", LINK_DELEGATED_2772)

                miss_61ae = _annex_61ae_check()
                if miss_61ae:
                    local_missing.append(f"Annex I 61 missing: ({_format_letter_list(miss_61ae)})")

                status = "Completed" if len(local_missing) == 0 else "Incomplete"
                return status, "Option A: Member State applies FULL obligations (Article 4(4)).", local_missing

            def evaluate_rbc_readiness_for_option_b():
                """
                This function handles Option B logic:
                - Always full obligations: Article 4(1) + Article 4(2)
                """
                local_missing = []

                print("\nAccording to Article 4(1), you must cover ALL due-diligence elements in CSDDD Article 5(1)(a)-(g):")
                print(
                    """Article 5(1) actions:
(a) integrate due diligence into policies and risk management systems
(b) identify and assess actual/potential adverse impacts (and prioritise where needed)
(c) prevent/mitigate potential impacts; bring actual impacts to an end and minimise extent
(d) provide remediation for actual adverse impacts
(e) meaningful engagement with stakeholders
(f) notification mechanism + complaints procedure
(g) monitor effectiveness of due diligence policy and measures
"""
                )
                print("Link:", LINK_CSDDD)

                miss_ag = _csddd_ag_check()
                if miss_ag:
                    local_missing.append(f"CSDDD Article 5(1) missing: ({_format_letter_list(miss_ag)})")

                print("\nAlso required (according to Article 4(2)): publish a public statement covering Annex I point 61(a)-(e).")
                print(
                    """Core elements (Annex I 61(a)-(e)):
(a) embedding due diligence in governance, strategy and business model
(b) engaging with affected stakeholders
(c) identifying and assessing negative impacts on people and the environment
(d) taking action to address negative impacts on people and the environment
(e) tracking the effectiveness of these efforts
"""
                )
                print("Link:", LINK_DELEGATED_2772)

                miss_61ae = _annex_61ae_check()
                if miss_61ae:
                    local_missing.append(f"Annex I 61 missing: ({_format_letter_list(miss_61ae)})")

                status = "Completed" if len(local_missing) == 0 else "Incomplete"
                return status, "Option B: full due diligence + public statement.", local_missing


            # Here we actually run the correct function depending on Option A or B
            if option == "a":
                obligations_status, note, missing_items = evaluate_rbc_readiness_for_option_a()
            else:
                obligations_status, note, missing_items = evaluate_rbc_readiness_for_option_b()

            # ============================================================
            # RBC TOOLBOX — Step 3 — Results
            # ============================================================

            # Convert missing list into a single string for the table
            missing_text = "; ".join(missing_items) if missing_items else "-"

            # Create a 2-column table:
            # Left side = row names
            # Right side = values
            results_2col = pd.DataFrame(
                {
                    "Value": [
                        project_capacity_mw,
                        bidder_type_label,
                        obligations_status,
                        missing_text,
                    ]
                },
                index=[
                    "Project capacity (MW)",
                    "Bidder type",
                    "Obligations",
                    "Missing (if any)",
                ],
            )

            print("\n==================== RBC — RESULTS ====================")
            display(results_2col)

            # Final message
            if obligations_status == "Completed":
                print("\n✅ Result: You are READY for the RBC criterion and you have all mandatory documents.")
            else:
                print("\n❌ Result: Obligations are INCOMPLETE. Fix the missing items shown above to be ready for RBC.")







RBC toolbox (focus: only the RBC criterion)
Runs according to: Commission Implementing Regulation (EU) 2025/1176 (23 May 2025)
RBC is mandatory in this EU law with specific requirements. This toolbox follows exactly that.

Is the RBC criterion included in your auction? (yes/no): no

RBC is not included in your auction. You can skip this criterion.
